In [2]:
import pandas as pd
import json

# Load Excel sheets
dashboard = pd.ExcelFile("data.xlsx")
data = pd.read_excel(dashboard, "Statistics")
aes = pd.read_excel(dashboard, 'AEs')
referrals = pd.read_excel(dashboard, 'Referrals')

partnerDistricts =pd.read_excel("partnerDistrict.xlsx")

# Fill missing values and convert types
data.fillna(0, inplace=True)
aes.fillna(0, inplace=True)
referrals.fillna(0, inplace=True)

if type(referrals['ReferrelRecordingYear'])!= int:
    referrals['ReferrelRecordingYear'] = 2025

data['year'] = data['year'].astype(int)
data['recordingMonth'] = data['recordingMonth'].astype(str)
aes['AERecordingYear'] = aes['AERecordingYear'].astype(int)
aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)
referrals['ReferrelRecordingYear'] = referrals['ReferrelRecordingYear'].astype(int)
referrals['ReferrelRecordingMonth'] = referrals['ReferrelRecordingMonth'].astype(str)




# Process rows and build JSON
def process_row(row):
    row_data = row.to_dict()
    if int(pd.to_numeric(row.get('totalAES', 0), errors='coerce')) > 0:
        recordedAEs = aes[
            (aes['AERecordingSite'] == row['facilityName']) &
            (aes['AERecordingMonth'] == row['recordingMonth']) &
            (aes['AERecordingYear'] == row['year'])
        ]

        recordedAEs = recordedAEs[recordedAEs['ae_classification']!=0]

        row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
    else:
        row_data['matchingAES'] = []
            
    if int(pd.to_numeric(row.get('referralToOtherServices', 0), errors='coerce')) > 0:
        otherReferrals = referrals[
            (referrals['ReferrelRecordingSite'] == row['facilityName']) &
            (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
            (referrals['ReferrelRecordingYear'] == row['year'])
        ]

        otherReferrals = otherReferrals[otherReferrals['services_referred']!=0]
        # print(otherReferrals)
        row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
    else:
        print("no data")
        row_data['otherReferrals'] = []

    return row_data

partner_lookup = dict(zip(partnerDistricts['DISTRICT'], partnerDistricts['PARTNER']))

data['Partner'] = data['District'].map(partner_lookup)


json_data = data.apply(process_row, axis=1).tolist()

# Save to JSON
output_file = "data.json"
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(json.dumps(json_data, indent=4))



no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
[
    {
        "AERecordingMonth": 0,
        "AERecordingSite": 0,
        "AERecordingYear": 0,
        "District": "MARONDERA",
        "ReferrelRecordingMonth": 0,
        "ReferrelRecordingSite": 0,
        "ReferrelRecordingYear": 0,
        "Site_Name": "MARONDERA

In [3]:
data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,aeComments,...,total_mcs_referred_to_other_services,total_surgicalDisposable,total_surgicalReusable,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,vmmc_number,year,facilityType,Partner
0,0,0,0,MARONDERA,0,0,0,MARONDERA - 100903 - PROVINCIAL HOSPITAL,12,0,...,0,OK,OK,0,0,0.0,0,2025,0,ZACH
1,0,0,0,MARONDERA,0,0,0,MARONDERA - 100903 - PROVINCIAL HOSPITAL,10,0,...,0,OK,OK,0,0,0.0,0,2025,0,ZACH
2,0,0,0,MARONDERA,0,0,0,MARONDERA - 100903 - PROVINCIAL HOSPITAL,9,0,...,0,OK,OK,0,0,0.0,0,2025,0,ZACH
3,0,0,0,MARONDERA,0,0,0,MARONDERA - 100903 - PROVINCIAL HOSPITAL,18,0,...,0,OK,OK,0,0,0.0,0,2025,0,ZACH
4,0,0,0,MARONDERA,0,0,0,MARONDERA - 100903 - PROVINCIAL HOSPITAL,44,0,...,0,OK,OK,0,0,0.0,0,2025,0,ZACH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,33,0,...,0,OK,0,11,0,0.0,0,2025,Static,ZACH
91,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,10,0,...,0,OK,0,9,0,0.0,0,2025,Outreach,ZACH
92,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,44,0,...,0,OK,0,37,0,0.0,0,2025,Outreach,ZACH
93,0,0,0,HURUNGWE,0,0,0,TENGWE - 101886 - CLINIC,0,0,...,0,0,0,0,0,0.0,0,2025,Outreach,ZACH
